# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
import numpy as np
import pandas as pd
# pa quitar warnings

import warnings
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt

# ols = minimos cuadrados ordinarios, regresion lineal
import statsmodels.api as sm
from statsmodels.formula.api import ols

# esto son distribuciones
from scipy.stats import ttest_rel  # con muestras dependientes
from scipy.stats import ttest_ind # con muestras independientes
from statsmodels.multivariate.manova import MANOVA
from scipy.stats import f_oneway, norm, f, chi2, chi2_contingency


KeyboardInterrupt: 

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [2]:
# Run this code:

salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [3]:
# Your code here:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
# Your code here:
salaries.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               33183 non-null  object 
 1   Job Titles         33183 non-null  object 
 2   Department         33183 non-null  object 
 3   Full or Part-Time  33183 non-null  object 
 4   Salary or Hourly   33183 non-null  object 
 5   Typical Hours      8022 non-null   float64
 6   Annual Salary      25161 non-null  float64
 7   Hourly Rate        8022 non-null   float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


In [5]:
valores_nulos_totales = salaries.isnull().sum()
valores_nulos_totales

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [6]:
# Your code here:
salaries["Salary or Hourly"].count()


33183

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [7]:
# Your code here:
lista_dep_employ = salaries.groupby("Department")["Job Titles"].count()
lista_dep_employ

Department
ADMIN HEARNG                39
ANIMAL CONTRL               81
AVIATION                  1629
BOARD OF ELECTION          107
BOARD OF ETHICS              8
BUDGET & MGMT               46
BUILDINGS                  269
BUSINESS AFFAIRS           171
CITY CLERK                  84
CITY COUNCIL               411
COMMUNITY DEVELOPMENT      207
COPA                       116
CULTURAL AFFAIRS            65
DISABILITIES                28
DoIT                        99
FAMILY & SUPPORT           615
FINANCE                    560
FIRE                      4641
GENERAL SERVICES           980
HEALTH                     488
HUMAN RELATIONS             16
HUMAN RESOURCES             79
INSPECTOR GEN               87
LAW                        407
LICENSE APPL COMM            1
MAYOR'S OFFICE              85
OEMC                      2102
POLICE                   13414
POLICE BOARD                 2
PROCUREMENT                 92
PUBLIC LIBRARY            1015
STREETS & SAN             21

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [8]:
from scipy.stats import t

from scipy.stats import ttest_1samp 

from scipy.stats import ttest_rel   # muestras dependientes

from scipy.stats import ttest_ind   # muestras independientes

In [9]:
# datos del enunciado

mu =   30      # media poblacional, es decir, yo solo tengo una muestra que es el DF

mu_hat = 32.788558     # media muestral, media del experimento

std_hat =  12.112573   # std muestral, std del experimento

n = 8022            # tamaño de la muestra

In [10]:
# grados de libertad, tenemos una muestra de valores independientes

dof = n-1

dof

8021

In [11]:
# a manita

t_test = (mu_hat - mu) / (std_hat/np.sqrt(n))

t_test

20.61980734597066

In [12]:
p_value = t(df=dof).cdf(t_test)

p_value

1.0

In [13]:
help(t)

Help on t_gen in module scipy.stats._continuous_distns:

<scipy.stats._continuous_distns.t_gen object>
    A Student's t continuous random variable.
    
    For the noncentral t distribution, see `nct`.
    
    As an instance of the `rv_continuous` class, `t` object inherits from it
    a collection of generic methods (see below for the full list),
    and completes them with details specific for this particular distribution.
    
    Methods
    -------
    rvs(df, loc=0, scale=1, size=1, random_state=None)
        Random variates.
    pdf(x, df, loc=0, scale=1)
        Probability density function.
    logpdf(x, df, loc=0, scale=1)
        Log of the probability density function.
    cdf(x, df, loc=0, scale=1)
        Cumulative distribution function.
    logcdf(x, df, loc=0, scale=1)
        Log of the cumulative distribution function.
    sf(x, df, loc=0, scale=1)
        Survival function  (also defined as ``1 - cdf``, but `sf` is sometimes more accurate).
    logsf(x, df, loc=0

In [14]:
# Your code here:
hourly_data = salaries[salaries["Salary or Hourly"] == "Hourly"]

# saco el resumen estadístico de la columna 
hourly_stats = hourly_data.describe()
hourly_stats

,Typical Hours,Annual Salary,Hourly Rate
count,8022.000000,0.0,8022.000000
mean,34.507604,NaN,32.788558
std,9.252077,NaN,12.112573
min,10.000000,NaN,2.650000
25%,20.000000,NaN,21.200000
50%,40.000000,NaN,35.600000
75%,40.000000,NaN,40.200000
max,40.000000,NaN,109.000000


In [15]:
from scipy.stats import ttest_1samp

In [16]:
t_test = ttest_1samp(hourly_data["Hourly Rate"], mu, alternative='two-sided')

t_test

TtestResult(statistic=20.6198057854942, pvalue=4.3230240486229894e-92, df=8021)

In [17]:
t_test.pvalue

4.3230240486229894e-92

Rechazamos la hipotesis nula, ya que P_value es menor que 0,05% (corresponde a una confianza del 95%). Por tanto no son iguales 

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [28]:
# Your code here:
salary_data = salaries[salaries["Department"] == "POLICE"]

# saco el resumen estadístico de la columna 
salary_stats = salary_data.describe()
salary_stats


,Typical Hours,Annual Salary,Hourly Rate
count,10.0,13404.000000,10.000000
mean,20.0,86486.414503,9.384000
std,0.0,18272.228294,0.098116
min,20.0,38376.000000,9.270000
25%,20.0,84054.000000,9.270000
50%,20.0,90024.000000,9.460000
75%,20.0,94524.000000,9.460000
max,20.0,260004.000000,9.460000


In [29]:
# datos del enunciado

mu =   86000      # media poblacional, es decir, yo solo tengo una muestra que es el DF

n = 13404            # tamaño de la muestra

In [32]:
t_test = ttest_1samp(salary_data["Annual Salary"].dropna(), mu, alternative='greater')

t_test

TtestResult(statistic=3.081997005712994, pvalue=0.0010301701775482569, df=13403)

In [33]:
t_test.pvalue 

0.0010301701775482569

Estadisticamente el policia tiene razon

Using the `crosstab` function, find the department that has the most hourly workers. 

In [34]:
# Your code here:

cross = pd.crosstab(salaries[salaries['Salary or Hourly'] == 'Hourly']['Department'] , salaries[salaries['Salary or Hourly'] == 'Hourly']['Name'].count())
cross
# El departamento STREETS & SAN 1862

col_0,8022
Department,
ANIMAL CONTRL,19
AVIATION,1082
BUDGET & MGMT,2
BUSINESS AFFAIRS,7
CITY COUNCIL,64
COMMUNITY DEVELOPMENT,4
CULTURAL AFFAIRS,7
FAMILY & SUPPORT,287
FINANCE,44


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [46]:
# Your code here:

# datos del enunciado

mu =   35     # media poblacional, es decir, yo solo tengo una muestra que es el DF

n = 1862            # tamaño de la muestra


In [47]:
department_data = salaries[salaries["Department"] == "STREETS & SAN"]

# saco el resumen estadístico de la columna 
department_data_stats = department_data.describe()
department_data_stats

,Typical Hours,Annual Salary,Hourly Rate
count,1862.000000,336.000000,1862.000000
mean,37.137487,83019.250000,33.728378
std,8.777897,21254.991182,5.735242
min,10.000000,31872.000000,19.500000
25%,40.000000,70056.000000,35.600000
50%,40.000000,84168.000000,35.600000
75%,40.000000,97740.000000,36.210000
max,40.000000,154428.000000,53.100000


In [49]:
t_test = ttest_1samp(department_data["Hourly Rate"].dropna(), mu, alternative='less')

t_test

TtestResult(statistic=-9.567447887848152, pvalue=1.6689265282353859e-21, df=1861)

El valor es cercano a cero por lo que podemos rechazar la hipotesis nula

# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [55]:
from scipy.stats import sem   # error standard

In [56]:
hourly_data = salaries[salaries["Salary or Hourly"] == "Hourly"]

# saco el resumen estadístico de la columna 
hourly_stats = hourly_data.describe()
hourly_stats

,Typical Hours,Annual Salary,Hourly Rate
count,8022.000000,0.0,8022.000000
mean,34.507604,NaN,32.788558
std,9.252077,NaN,12.112573
min,10.000000,NaN,2.650000
25%,20.000000,NaN,21.200000
50%,40.000000,NaN,35.600000
75%,40.000000,NaN,40.200000
max,40.000000,NaN,109.000000


In [58]:
# Your code here:

# 95% de confianza de que la media este en ese intervalo


t.interval(0.95,                  # alfa = 1 - pvalor, confianza
           len(hourly_data["Hourly Rate"])-1,      # N-1, dof
           loc=hourly_data["Hourly Rate"].mean(),
           scale=sem(hourly_data["Hourly Rate"])
          )


(32.52345834488425, 33.05365708767623)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [66]:
salary_data = salaries[salaries["Department"] == "POLICE"]

cleaned_salary_data = salary_data["Annual Salary"].dropna()

# saco el resumen estadístico de la columna 
salary_stats = salary_data.describe()
salary_stats


,Typical Hours,Annual Salary,Hourly Rate
count,10.0,13404.000000,10.000000
mean,20.0,86486.414503,9.384000
std,0.0,18272.228294,0.098116
min,20.0,38376.000000,9.270000
25%,20.0,84054.000000,9.270000
50%,20.0,90024.000000,9.460000
75%,20.0,94524.000000,9.460000
max,20.0,260004.000000,9.460000


In [67]:
# Your code here:


t.interval(0.95,                  # alfa = 1 - pvalor, confianza
           len(cleaned_salary_data)-1,      # N-1, dof
           loc=cleaned_salary_data.mean(),
           scale=sem(cleaned_salary_data)
          )

(86177.05631531784, 86795.77269094894)

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
%pip install statsmodels
%pip install -U scikit-learn

In [52]:
from statsmodels.stats.proportion import proportions_ztest

In [53]:
# Your code here:

# ¿la proporcion de a es distinta del 25%?

# H0 : No, es igual
# H1 : Si, es distinta la proporcion



z_score, p_value= proportions_ztest(salaries['Salary or Hourly'].value_counts()["Hourly"],   # 25 elementos de a, 
                                      len(salaries['Salary or Hourly']),                     # nº total de elementos
                                      0.25                                                   # proporcion
                                     )


z_score, p_value

(-3.5099964213703005, 0.0004481127249057967)

In [54]:
p_value

0.0004481127249057967